In [ ]:
import pandas as pd
import numpy as np
data=pd.read_csv('')
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data.isin(['',' ']).sum()
data['TotalCharges']=pd.to_numeric(data['TotalCharges'],errors='coerce')
#when u convert to numeric blank spaces will be error , the coerce will fill
#with NaN
import seaborn as sns
import matplotlib.pyplot as plt
sns.histplot(data['TotalCharges'],kde=True)
plt.show()

In [ ]:
from sklearn.impute import SimpleImputer
si=SimpleImputer(strategy='median')
data['TotalCharges']=si.fit_transform(data[['TotalCharges']])


In [ ]:
from sklearn.preprocessing import LabelEncoder
col_names=data.select_dtypes('object').columns
for col in col_names:
    lbl=LabelEncoder()
    data[col]=lbl.fit_transform(data[col])
data.info()

In [ ]:
from sklearn.preprocessing import StandardScaler
X=data.drop(['customerID','Churn'],axis=1)
Y=data['Churn']
sc=StandardScaler()
X_Scaled=sc.fit_transform(X)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import accuracy_score
bg_model=BaggingClassifier(estimator=DecisionTreeClassifier(),
                           n_estimators=100,
                          bootstrap=True,
                          oob_score=True,max_samples=0.8,
                          random_state=42)
X_train,X_test,Y_train,Y_test=train_test_split(X_Scaled,Y,
                                              test_size=0.2,random_state=42)
bg_model.fit(X_train,Y_train)
y_pred=bg_model.predict(X_test)
print('Accuracy Score is:',accuracy_score(Y_test,y_pred))
print('Out of Box Score is:',bg_model.oob_score_)
score=cross_val_score(DecisionTreeClassifier(),X,Y,cv=5)
print(score.mean())
bg_score=cross_val_score(bg_model,X,Y,cv=5)
print(bg_score.mean())


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gradient_boost_model=GradientBoostingClassifier(n_estimators=100,
                                               learning_rate=0.1,
                                               max_depth=3,random_state=42)
gradient_boost_model.fit(X_train,Y_train)
y_gb_predict=gradient_boost_model.predict(X_test)
print('Accuracy is:',accuracy_score(Y_test,y_gb_predict))
#n_estimators - number of boosting stages
#learning_rate->shrinks the contribution of each tree so the learning rate to avoid
#trade off between LR and the n_estimatore this is low value
#max_depth=it is used to control the depth of the tree


In [ ]:
#stacking Classifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from sklearn.ensemble import StackingClassifier
base_models=[('rf',RandomForestClassifier(n_estimators=100)),
             ('dt',DecisionTreeClassifier()),
             ('SVC',SVC(kernel='rbf'))
            ]
meta_model=GradientBoostingClassifier()
stacking_model=StackingClassifier(estimators=base_models,
                                 final_estimator=meta_model,
                                 cv=5,#cross validation by defualt None u 5 or 10
                                 stack_method='auto')
stacking_model.fit(X_train,Y_train)
y_pred_stack=stacking_model.predict(X_test)
print('Accuracy is:',accuracy_score(Y_test,y_pred_stack))



StackingClassifier(
    estimators,#base model
    final_estimator=None,#meta model
    *,
    cv=None,
    stack_method='auto',
    n_jobs=None,
    passthrough=False,
    verbose=0,
)